# Steiner Trees on Road Maps
This example retrieves a graph from [OpenStreetMap](https://www.openstreetmap.org) via the [osmnx](https://osmnx.readthedocs.io) package.
A shortest tree connecting a set of randomly chosen terminals is then found by computing a minimum weight [Steiner tree](https://en.wikipedia.org/wiki/Steiner_tree_problem#Steiner_tree_in_graphs_and_variants).

## Imports

In [ ]:
import random

import networkx as nx

import osmnx as ox
import pyproj

from graphilp.imports import networkx as imp_nx
from graphilp.network import steiner_linear_tightened as stp
from graphilp.network.heuristics import steiner_metric_closure as smc

## Load road map graph

Use [Gauss-Krüger 3](https://georepository.com/crs_31467/DHDN-3-degree-Gauss-Kruger-zone-3.html) as coordinate system

In [ ]:
crs = pyproj.crs.CRS('epsg:31467')

In [ ]:
place = 'Oberkassel, Düsseldorf, Deutschland'

# road network of suburb (converted to Gauss-Krüger 3)
G = ox.project_graph(ox.graph_from_place(place, network_type='walk'), to_crs=crs)

In [ ]:
# draw the road map
ox.plot_graph(G, figsize=(14, 10),
    bgcolor='#FFF',
    node_color='b');

# Choose terminals

In [ ]:
node_list = list(G.nodes())

In [ ]:
# choose num_terminals random terminals
num_terminals = 5

In [ ]:
terminals = [node_list[random.randint(0, len(node_list))] for n in range(num_terminals)]

In [ ]:
# draw road map and terminals
ox.plot_graph(G, figsize=(14, 10),
    bgcolor='#FFF',
    node_color=['#ED0000' if n in terminals else '#00F' for n in node_list],
    node_size=[50 if n in terminals else 15 for n in node_list]);

## Set up and run optimisation problem

In [ ]:
# transform input into an undirected graph
G_undirected = nx.Graph(G)

In [ ]:
# import to GraphILP API
optG = imp_nx.read(G_undirected)

In [ ]:
# find a heuristic solution
warmstart, lower_bound = smc.get_heuristic(optG, terminals)

In [ ]:
# set up model and warmstart
m = stp.create_model(optG, terminals, weight='length', warmstart=warmstart)

In [ ]:
m.optimize()

## Visualise solution

In [ ]:
# extract solution
solution = stp.extract_solution(optG, m)

In [ ]:
# draw road map with terminals and solution

In [ ]:
edge_colors = ['#ED0000' if (u,v) in solution or (v, u) in solution else '#AAA' for u, v in G.edges()]
edge_widths = [3 if (u,v) in solution else 1 for u, v in G.edges()]

In [ ]:
ox.plot_graph(G, figsize=(14, 10),
    bgcolor='#FFF',
    node_color=['#ED0000' if n in terminals else '#00F' for n in node_list],
    node_size=[75 if n in terminals else 15 for n in node_list],
             edge_color=edge_colors, edge_linewidth=edge_widths);